# Classical and relativistic hydrodynamics

This notebook shows how to formulate and numerically solve the equations of classical and special-relativistic hydrodynamics in 1D, assuming a slab geometry.

We will use a custom python-3 library for the solution of balance laws in 1D, which is implemented using [numpy](http://www.numpy.org/). This notebook also uses [matplotlib](https://matplotlib.org/) and [ipywidgets](https://ipywidgets.readthedocs.io/en/latest/) for the visualization and the root finding routines from [scipy](https://www.scipy.org/). All of these packages are easily available through the [anaconda python distribution](https://anaconda.org/anaconda/python).

In [ ]:
from ipywidgets import interact, FloatSlider
import matplotlib.pyplot as plt
from math import fabs, sqrt
import numpy as np
import PyHRSC1D as hrsc
from scipy.optimize import bisect, brentq

In [ ]:
import matplotlib as mpl

%matplotlib inline

## Newtonian hydrodynamics

We start with the Newtonian hydrodynamics

### Basic equations

The classical equations of hydrodynamics describe the conservation of mass, momentum, and energy in a fluid. In one spatial dimension they are

$$
\begin{align}
    & \partial_t \rho + \partial_x (\rho v) = 0, \\
    & \partial_t S + \partial_x (S v + p) = 0, \tag{1}  \\
    & \partial_t E + \partial_x [(E + p) v] = 0;
\end{align}
$$

where $\rho$ is the density, $v$ the velocity, $S = \rho v$ the linear momentum, $p$ the pressure, $E = \rho \epsilon + \frac{1}{2} \rho v^2$ is the total energy density, and $e$ is the specific internal energy.

Note that these equations need to be closed with the choice of an equation of state $p = p(\rho, \epsilon)$. Here we will use the simple ideal-gas equation of state

$$
    p = (\gamma - 1) \rho \epsilon,
$$

where $\gamma$ is the *adiabatic index*.

In the case of smooth flows, when shocks are not present, the last equation in (1) can be replaced by the equation of entropy conservation

$$
\partial_t (s \rho) + \partial_x (s \rho v) = 0, \tag{2}
$$

where $s$ is the specific entropy of the fluid (not to be confused with the linear momentum $S$ above). When shock waves are present in the flow, it is necessary to solve Eq. (1), which remains valid in the integral sense even when shocks are present.


### Conservation form

They are a particular case of a general family of equations called *balance laws*

$$
    \partial_t \mathbf{F}^0(\mathbf{u}) +
        \partial_x \mathbf{F}^x(\mathbf{u}) =
        \mathbf{S}(\mathbf{u}), \tag{3}
$$

where

$$
    \mathbf{u} = \begin{pmatrix}
        \rho \\
        v \\
        \epsilon
    \end{pmatrix},
    \quad
    \mathbf{F}^0 = \begin{pmatrix}
        \rho \\
        S \\
        E
    \end{pmatrix}\textrm{, and}
    \quad
    \mathbf{F}^x = \begin{pmatrix}
        \rho v \\
        S v  + p \\
        (E + p) v
    \end{pmatrix},
$$

are respectively the *primitive variables*, *conservative variables*, and the *fluxes*. The *source terms* for the Euler equation are simply $\mathbf{S} = 0$.

There is a vast literature on how to solve equations in the form (3) numerically. Here, we are going to use a variant of the so-called [Kurganov-Tadmor central scheme](https://dx.doi.org/10.1006/jcph.2000.6459). The numerical solver implemented in the `PyHRSC1D` library can handle any equation in the form (3), to specialize it to the case of the Euler equations we need to provide the following functions

$$
    \texttt{prim2con}: \mathbf{u} \mapsto \mathbf{F}^0,
    \qquad
    \texttt{con2prim}: \mathbf{F}^0 \mapsto \mathbf{u},
    \qquad
    \texttt{fluxes}: \mathbf{u} \mapsto \mathbf{F}^x.
$$



`PyHRSC1D` will also need an estimate of the propagation velocities of hydrodynamic waves. For the hydrodynamics equations the characteristic waves are the material waves, propagating with velocity $v$, and the sound waves, which propagate with velocity $c_s$ in the frame coming with the fluid ($c_s$ depends on the equation of state). So the characteristic velocities are

$$
    \lambda_\pm = v \pm c_s, \qquad \lambda_c = v.
$$

### Sound waves

#### Linearized equations

The dispersion relation for sound waves can be computed from the dispersion relation of **isentropic** perturbations of Eq. (1) on a constant background $\rho = \mathrm{const}$, $v = 0$. Since we assume an isentropic flow, we can drop the energy conservation equation from Eq (1), since it is equivalent to Eq. (2) which is trivially satisfied. Keeping only the linear terms we find

$$
\begin{align}
    & \partial_t (\delta\rho) + \rho \partial_x (\delta v) = 0, \\
    & \rho \partial_t (\delta v) + \partial_x (\delta p) = 0.
\end{align}
$$

If we derive the first equation in time and the second in space we have

$$
\begin{align}
    & \partial_t^2 (\delta\rho) + \rho \partial_t (\partial_x \delta v) = 0, \\
    & \rho \partial_x \partial_t (\delta v) + \partial_x^2 (\delta p) = 0,
\end{align}
$$

substituting $\rho \partial_t \partial_x \delta v$ in the first equation using the second we find

$$
    \partial_t^2 (\delta\rho) - \partial_x^2 (\delta p) = 0.
$$

Using the chain rule and assuming isentropic perturbations we can write

$$
    \delta p = \left(\frac{\partial p}{\partial \rho}\right)_s \delta \rho =: c_s^2 \delta \rho \tag{4}
$$

Finally, we find that the density fluctuations propagate according to the wave equation with velocity $c_s$ (note that we are working at linear order, so we can neglect changes in $c_s$ due to the fluctuations):

$$
    \partial_t^2 (\delta\rho) - c_s^2 \partial_x^2 (\delta \rho) = 0
$$

#### Thermodynamical relations

To evaluate Eq. (4), we need to express the derive the expression for the pressure as a function of density for *isentropic* fluctuations.

$$
    \mathrm{d} \epsilon = p \frac{\mathrm{d} \rho}{\rho^2}, \tag{5}
$$

using the equation of state we find

$$
    \epsilon = \frac{p}{(\gamma - 1) \rho}.
$$

Using this relation we can rewrite Eq. (5) as

$$
    \frac{1}{\gamma-1} \mathrm{d}\left( \frac{p}{\rho} \right) = p \frac{\mathrm{d}\rho}{\rho^2}, \tag{6}
$$

which, after some manipulation yields

$$
    \gamma \frac{\mathrm{d} \rho}{\rho} = \frac{\mathrm{d} p}{p},
$$

that implies

$$
    p = K \rho^\gamma,
$$

where $K$ is an integration constant. Since the pressure must also satisfy the equation of state we find that

$$
    K = (\gamma - 1) \rho^{1 - \gamma} \epsilon.
$$

Finally we can write

$$
    c_s^2 = \left( \frac{\partial p}{\partial \rho} \right)_S
          = K \gamma \rho^{\gamma - 1} = (\gamma - 1) \gamma \epsilon
$$

### Implementation

First we define an helper class that evaluates the equation of state

In [ ]:
class IdealGas(object):
    """
    A simple ideal-gas EOS
    """
    def __init__(self, gamma=4./3.):
        self.gamma = gamma
    def press(self, rho, eps):
        """
        Computes the pressure given density and specific internal energy
        """
        return (self.gamma - 1.)*rho*eps
    def energy(self, rho, eps):
        """
        Computes the energy density given density and specific internal energy
        """
        return rho*eps
    def csound(self, rho, eps):
        """
        Computes the sound speed given density and specific internal energy
        """
        assert(np.all(eps >= 0))
        return np.sqrt((self.gamma - 1.)*self.gamma*eps)

`PyHRSC1D` requires that we specify the equations we are solving by specializing a generic conservation law class. We need to provide all of the methods listed below:

In [ ]:
help(hrsc.CLaw)

In [ ]:
class Euler(hrsc.CLaw):
    """
    Class implementing the classical hydrodynamics equations
    
    We use the following definitions
    
    prim : (rho, v, eps)
    cons : (D [= rho], S, E)
    """
    nvars = 3
    varnames = ["rho", "v", "eps"]
    def __init__(self, eos):
        self.eos = eos
    def prim2con(self, prim, cons, argv=None):
        # Some simple aliases
        rho, v, eps = prim
        D, S, E = cons
        # Compute the conservative variables
        D[:] = rho
        S[:] = rho*v
        E[:] = rho*(eps + 0.5*v**2)
    def con2prim(self, cons, prim, argv=None):
        rho, v, eps = prim
        D, S, E = cons
        rho[:] = D
        v[:] = S/D
        eps[:] = (E - 0.5*S*v)/D
    def speeds(self, prim, cons, char, argv=None):
        cs = self.eos.csound(prim[0], prim[2])
        v = prim[1]
        char[0,:] = v - cs
        char[1,:] = v
        char[2,:] = v + cs
    def fluxes(self, prim, cons, flux, argv=None):
        rho, v, eps = prim
        D, S, E = cons
        p = self.eos.press(rho, eps)
        flux[0,:] = rho*v
        flux[1,:] = S*v + p
        flux[2,:] = (E + p)*v
    def sources(self, prim, cons, source, argv=None):
        source[:] = 0.

### Example: Sod's shock tube

A classical hydrodynamics test is to consider two regions with two fluids initially at rest separated by a membrane at $x=0$. At $t=0$ the membrane is removed and the fluids interact. This is a so-called shock tube setup. The particular example we consider was originally proposed by Sod in a <a href="https://doi.org/10.1016/0021-9991(78)90023-2">classical paper</a>. The equation of state is with $\gamma = 1.4$ and the initial conditions are described by a left and right state given by

$$
    (\rho_L, v_L, \epsilon_L) = (1, 0, 2.5), \quad
    (\rho_R, v_R, \epsilon_R) = (0.125, 0, 2)
$$

With `PyHRSC1D` we need to create an initial data class to specify the initial conditions

In [ ]:
class SodInitialData(hrsc.InitDataGeneric):
    def apply(self, grid, prim, argv=None):
        """
        Set up the value of the primitive variables.
        """
        rho, v, eps = prim
        rho[:] = np.where(grid.xc < 0, 1.0, 0.125)
        v[:] = 0.
        eps[:] = np.where(grid.xc < 0, 2.5, 2.0)

We setup a uniform grid between $-0.5$ and $0.5$ with $N=100$ grid points. We adopt static boundary conditions, meaning that the value of the solution in the ghost regions is left unchanged. This is how we setup the solver:

In [ ]:
grid_sod = hrsc.make_uniform_grid(-0.5, 0.5, 100)
eos_sod = IdealGas(gamma=1.4)
hydro = Euler(eos_sod)
ic_sod = SodInitialData()
bc_sod = hrsc.BoundaryStatic()
solver_sod = hrsc.KurganovTadmorSolver(grid_sod, hydro, ic_sod, bc_sod)

To ensure the stability of the numerical integration we must ensure that

$$
    c_\max \Delta t \leq \Delta x
$$

where $c_\max$ is the maximum characteristic speed (the maximum between $|\lambda_\pm|$ and $|\lambda_c|$ defined above). To this aim we will set

$$
    \Delta t = \mathrm{CFL}\ \frac{\Delta x}{c_\max}
$$

where $\mathrm{CFL}$ is set to be

In [ ]:
CFL = 0.9

This is a helper script that will run a solver and save some of the data frames. In a real code, we would write this data to disk, to avoid filling up the memory with stuff, but for a 1D problem we can afford to store a lot of data.

In [ ]:
class NumericalSolution(object):
    """
    Helper class for storing the numerical solution
    """
    def __init__(self):
        self.times  = []
        self.frames = []
    def save(self, time, varnames, prim):
        self.times.append(time)
        self.frames.append({})
        for name, ivar in zip(varnames, range(prim.shape[0])):
            self.frames[-1][name] = prim[ivar].copy()

def run_solver(solver, CFL=0.9, t_start=0., t_end=1., nframes=2):
    """
    Helper function to run simulations
    
    * solver  : a conservation law solver from PyHRSC
    * t_start : initial time of the simulation
    * t_end   : final time of the simulation
    * nframes : number of frames to save
    
    Returns
    
    The numerical solution at specific times
    """
    # This creates the initial data
    solver.initialize()
    
    if nframes > 1:
        dt_save = (t_end - t_start)/(nframes - 1)
    else:
        dt_save = np.inf
    t_save_next = dt_save
    solution = NumericalSolution()
    solution.save(t_start, solver.claw.varnames, solver.prim)
    
    t = t_start
    while t < t_end:
        dt = solver.estimate_timestep(CFL)
        if t + dt > t_save_next:
            dt = t_save_next - t
        elif t + dt > t_end:
            dt = t_end - t
        solver.step(dt)
        t += dt
        if t >= t_save_next:
            solution.save(t, solver.claw.varnames, solver.prim)
            print("Time: {}".format(t))
            t_save_next += dt_save
            
    return solution

In [ ]:
sol_sod = run_solver(solver_sod, CFL=CFL, t_start=0, t_end=0.2, nframes=10)
sol_sod.times = np.array(sol_sod.times)
for frame in sol_sod.frames:
    frame["p"] = eos_sod.press(frame["rho"], frame["eps"])

The next cell visualizes the results as a function of time. The solution consist of a rarefaction wave traveling to the left and two discontinuities traveling to the right. The rightmost is a shock wave, while the second one is called *contact discontinuity*, because it contains a jump in the density, but not in the pressure of the fluid.

In [ ]:
@interact(time=FloatSlider(value=0, min=sol_sod.times[0], max=sol_sod.times[-1],
                           step=sol_sod.times[1] - sol_sod.times[0],
                           continuous_update=True))
def plot_sod(time):
    idx = np.argmin(np.abs(sol_sod.times - time))
    plt.figure(figsize=[10,7])
    plt.title("Time: {:.3f}".format(sol_sod.times[idx]))
    plt.plot(grid_sod.xc, sol_sod.frames[idx]["rho"], ".", label=r"$\rho$", color="blue")
    plt.plot(grid_sod.xc, sol_sod.frames[idx]["v"], ".", label=r"$v$", color="red")
    plt.plot(grid_sod.xc, sol_sod.frames[idx]["p"], ".", label=r"$p$", color="green")
    plt.xlim(-0.5, 0.5)
    plt.ylim(-0.1, 1.1)
    plt.legend()
    plt.show()

The solution to the Riemann problem can be solved analytically. I have included a tabulated solution for the Sod problem in the folder `./data`. The code in the next cell plots it. Note how the contact discontinuity is smeared over many cells by the numerical code. This is unfortunately typical for central schemes like the Kurganov-Tadmor scheme we are using here.

In [ ]:
exact = np.load("./data/newt_sod_t0p2.npz")
plt.figure(figsize=[10,7])
plt.title("Time: 0.2")
plt.plot(exact["x"], exact["rho"], "-", label=r"$\rho$ exact", color="blue")
plt.plot(exact["x"], exact["v"], "-", label=r"$v$ exact", color="red")
plt.plot(exact["x"], exact["p"], "-", label=r"$p$ exact", color="green")
plt.plot(grid_sod.xc, sol_sod.frames[-1]["rho"], ".", label=r"$\rho$ numerical", color="blue")
plt.plot(grid_sod.xc, sol_sod.frames[-1]["v"], ".", label=r"$v$ numerical", color="red")
plt.plot(grid_sod.xc, sol_sod.frames[-1]["p"], ".", label=r"$p$ numerical", color="green")
plt.legend()

## Relativistic hydrodynamics

We now switch to the equations of special-relativistic hydrodynamics. For simplicity, we will work in units in which $c = 1$.

### Basic equations

The special relativistic hydrodynamics equation describe the conservation of energy and momentum of a fluid:

$$
    \partial_\nu T^{\mu\nu} = 0. \tag{7}
$$

We consider an ideal fluid where $T^{\mu\nu}$ is given by

$$
    T^{\mu\nu} = (\rho + p) u^\mu u^\nu + p \eta^{\mu\nu}, \tag{8}
$$

where $\rho$ is the **energy** density, $p$ is the pressure, $u^\mu$ the fluid four-velocity, and $\mathbf{\eta} = \mathrm{diag}(-1, 1, 1, 1)$ is the Minkowsky's metric. 

At the energy scales relevant for neutron star mergers we can assume the baryon number to be conserved. We introduce the number density current $J^\mu = n u^\mu$. The conservation of the total number of baryons reads

$$
    \partial_\mu J^\mu = 0. \tag{9}
$$

In analogy with the classical case it is convenient to split the energy density in a term due to the rest-mass contribution and a term due to the thermal energy

$$
    \rho = n\, m_b\, (c^2 + \epsilon), \tag{10}
$$

where we have introduced a mass scale $m_b$ and temporarily restored the speed of light $c (= 1)$ for clarity. $\epsilon$ is the specific internal energy. We also introduce the rest-mass density (not to be confused with the energy density $\rho$):

$$
    \rho_0 = n\, m_b.
$$

The equation of state we will consider for relativistic hydrodynamics is that of an ideal gas

$$
    p = m_b\, (\gamma - 1)\, n\, \epsilon = (\gamma - 1)\, \rho_0\, \epsilon , \tag{11}
$$

In the case of smooth relativistic flows (for a perfect fluid with no viscosity), as was the case for Newtonian flows, it is possible to show the law of entropy conservation holds:

$$
    \partial\mu ( n\, s\, u^\mu ) = 0,
$$

where $s$ is the entropy per baryon.

### Formulation as balance law

These equation can be brought to the form of Eq. (3) with the choice

$$
    \mathbf{u} = \begin{pmatrix}
        \rho_0 \\ u^x \\ \epsilon
    \end{pmatrix},
    \qquad
    \mathbf{F}^0 = \begin{pmatrix}
        D \\ S \\ E
    \end{pmatrix},
    \qquad
    \mathbf{F}^i = \begin{pmatrix}
        D v \\ S v + p \\ (E + p) v
    \end{pmatrix}, \tag{12}
$$

where $v$ is the spatial velocity

$$
    v = \frac{\mathrm{d} x}{\mathrm{d} t}
      = \frac{\mathrm{d} x}{\mathrm{d} \tau}
          \left(\frac{\mathrm{d} t}{\mathrm{d} \tau}\right)^{-1}
      = u^x \left( u^0 \right)^{-1}
$$

and we have introduced the conservative variables

$$
    D = \rho_0 W, \qquad S = (\rho + p) W^2 v, \qquad E = (\rho + p) W^2 - p,\tag{13}
$$

In the previous equations we have also used $W$ for the Lorentz factor:

$$
    W = u^0 = \frac{1}{\sqrt{1 - v^2}}.
$$

Note that Eq. (12) is very similar to the classical equations. The two most important differences are
- The appearance of the Lorentz factor $W$, which introduces a new non-linearity to the equations
- The fact that energy density and pressure now contribute to the momentum (or inertia) of the fluid

#### Recovery of the primitive variables

Many methods have been proposed and used throughout the literature to handle the recovery of the primitive variables. Here, we proceed as follows. Given an initial guess for the pressure $\tilde{p}$, then we can find the primitive variables as follows

$$
\begin{align}
    & v = \frac{S}{E + \tilde{p}}, \\
    & W = (1 - v^2)^{-1/2}, \\
    & \rho_0 = \frac{D}{W}, \\
    & \epsilon = \frac{E - D W + \tilde{p} (1 - W^2)}{D W}.
\end{align}
$$

We can use these values to compute the pressure from the equation of state and compare it with the pressure guess. The problem then reduces to the rootfinding of the function:

$$
    f(\tilde{p}) = p\left(\rho_0(\tilde{p}), \epsilon(\tilde{p})\right) - \tilde{p}.\tag{14}
$$

To use a bracketing method we construct an upper limit for $p$ under the additional assumption that $p(n, \epsilon)$ is a monotonically increasing function of $n$ and $\epsilon$. This assumption is verified by all reasonable equations of state, including the ideal-gas equation of state we are using. An upper limit on $\rho_0$ follows from the definition of $D$:

$$
    D = \rho_0 W \geq \rho_0
$$

For $\epsilon$ we have

$$
    \epsilon = \frac{E - D W + \tilde{p} (1 - W^2)}{D W} \leq \frac{E}{DW},
$$

so we can set

$$
    p_\max = p\left(D, \frac{E}{DW}\right)\tag{15}.
$$

As lower limit on $p$ we simply take $p_{\min} := 0$.

Summarizing. The recovery of the primitive variables consist of finding a root for Eq. (14), in the interval $[p_\min, p_\max]$. Each evaluation of Eq. (14) involves computing the primitives from a trial guess for the pressure and calling the equation of state.

### Sound waves

#### Linearized equations

We proceed as in the Newtonian case and linearize the equations around the equilibrium solution $n \equiv \mathrm{const}$, $u^0 = 1$, $u^x = 0$. It is useful to note that, because of the normalization condition of the velocity we have

$$
    -1 = -(u^0)^2 + (u^x)^2 \implies u^0 = \sqrt{(u^x)^2 + 1}.
$$

Consequently, in the case of a static background with zero velocity in the $x$ direction

$$
    \delta u^0 = \frac{1}{2} \frac{2 u^x \delta u^x}{\sqrt{(u^x)^2 + 1}} = 0.
$$

The linearized equations for small perturbations in the number density and momentum read

$$
\begin{align}
    & \partial_t (\delta n) + n \partial_x (\delta u^x) = 0 \\
    & (\rho + p) \partial_t (\delta v) + \partial_x (\delta p) = 0.
\end{align}
$$

We introduce the enthalpy per baryon $h = (\rho + p)/n$. Proceeding like in the Newtonian case we find

$$
    \partial_t^2 (\delta n) - \frac{1}{h} \partial_x^2 (\delta p) = 0,
$$

from which we infer that the relativistic sound speed can be computed as

$$
    h c_s^2 = \left(\frac{\partial p}{\partial n}\right)_s.\tag{17}
$$

Note the appearance of a new factor $h$ in the definition of the sound speed, compared to the Newtonian case.

Another possibility is to linearize the energy equation, instead of the number density conservation equation, to find

$$
    \partial_t^2 (\delta \rho) - \partial_x^2 (\delta p) = 0,
$$

Which shows that the sound speed can also be computed as

$$
    c_s^2 = \left(\frac{\partial p}{\partial \rho}\right).
$$

#### Characteristic velocity

As in the Newtonian case, the characteristic waves are given by the material waves, traveling with velocity $v$, and the sound waves, propagating with celerity $c_s$ in the fluid frame. From the Lorentz formula for the addition of velocities we find

$$
    \lambda_\pm = \frac{v \pm c_s}{1 \pm v c_s}, \quad \lambda_c = v
$$

#### Thermodynamics identities

In the case of relativistic fluids the rest mass enters the first law of thermodynamics which may be written in terms of energy per baryon as

$$
    \mathrm{d}\left(\frac{\rho}{n}\right) =
        T \mathrm{d}s - p \mathrm{d}\left(\frac{1}{n}\right), \tag{18}
$$

where $s$ is the entropy per baryon. For an isentropic flow we can proceed as in the Newtonian case to find from Eqs. (10) and (11):

$$
    \mathrm{d}\left(\frac{\rho}{n}\right) = m_b \mathrm{d}\epsilon =
        \frac{1}{\gamma -1} \mathrm{d}\left(\frac{p}{n}\right)
$$

Substituting this back into Eq. (18) and using $\mathrm{d}s = 0$, we find

$$
    \frac{1}{\gamma - 1} \mathrm{d} \left(\frac{p}{n}\right) =
        p \frac{\mathrm{d} n}{n^2}
$$

This is formally identical to Eq. (6) with $n$ instead of $\rho$ and yields

$$
    p = K n^\gamma,
$$

where

$$
    K = m_b\, (\gamma - 1)\, n^{1-\gamma}\, \epsilon
$$

must be constant for an isentropic flow.

### Implementation

As with the Newtonian hydrodynamics, we start with an helper class for the equation of state

In [ ]:
class RelIdealGas(object):
    """
    A relativistic version of the ideal-gas EOS
    """
    def __init__(self, gamma=4./3.):
        self.gamma = gamma
    def press(self, rho0, eps):
        """
        Computes the pressure given density and specific internal energy
        """
        return (self.gamma - 1.)*rho0*eps
    def energy(self, rho0, eps):
        """
        Computes the energy density given density and specific internal energy
        """
        return rho0*(1. + eps)
    def csound(self, rho0, eps):
        """
        Computes the sound speed given density and specific internal energy
        """
        return np.sqrt(((self.gamma - 1.)*self.gamma*eps)/(self.gamma*eps + 1.))

It is usually a good idea to split out the `con2prim` to a separate kernel. For large calculations, it will be necessary to switch to a low-level C implementation of this kernels, but for the examples we are considering here, a simple python implementation is sufficient.

In [ ]:
def con2prim_given_ptilde(p, D, S, E):
    try:
        v    = S/(E + p)
        W    = 1.0/sqrt(1 - v*v)
        ux   = W*v
        rho0 = D/W
        eps  = (E - D*W + p*(1 - W*W))/(D*W)
    except ValueError:
        print((p, D, S, E))
        raise
    return (rho0, ux, eps)

def con2prim_root_func(ptilde, eos, D, S, E):
    """
    Function defining the con2prim search
    
    * ptilde    : [in] trial pressure in a point
    * eos       : [in] equation of state handle
    * D, S, E   : [in] conservative varibales in a point
    """
    rho0, ux, eps = con2prim_given_ptilde(ptilde, D, S, E)
    return eos.press(rho0, eps) - ptilde

def con2prim_kernel(eos, D, S, E, rho0, ux, eps):
    """
    Simple implementation of the con2prim using scipy
    
    * eos           : [in] equation of state handle
    * D, S, E       : [in] conservative varibales vectors
    * rho0, ux, eps : [out] primitive variables vectors
    """
    # Check that arguments make sense
    assert(len(D.shape) == 1)
    assert(D.shape == S.shape == E.shape)
    assert(rho0.shape == ux.shape == eps.shape)
    assert(rho0.shape == D.shape)
    # Loop over grid point and invert the conservatives
    for i in range(D.shape[0]):
        rho0_max = D[i]
        eps_max = E[i]/D[i]
        p_min = 1e-15 # small_eps
        p_max = eos.press(rho0_max, eps_max)
        pnew = brentq(con2prim_root_func, p_min, p_max, args=(eos, D[i], S[i], E[i]))
        rho0[i], ux[i], eps[i] = con2prim_given_ptilde(pnew, D[i], S[i], E[i])

This is the main class implementing the relativistic Euler equations

In [ ]:
def W_v_from_ux(ux):
    """
    A helper function to convert from ux to v and W
    """
    W = np.sqrt(ux*ux + 1.)
    v = ux/W
    return W, v
    
class RelEuler(hrsc.CLaw):
    """
    Class implementing the relativistic hydrodynamics equations
    
    We use the following definitions
    
    prim : (rho0, ux, eps)
    cons : (D, S, E)
    """
    nvars = 3
    varnames = ["rho0", "ux", "eps"]
    def __init__(self, eos):
        self.eos = eos
    def prim2con(self, prim, cons, argv=None):
        D, S, E = cons
        rho0, ux, eps = prim
          
        W, v = W_v_from_ux(ux)
        
        p = self.eos.press(rho0, eps)
        rho = self.eos.energy(rho0, eps)
        
        D[:] = rho0*W
        S[:] = (p + rho)*W*W*v
        E[:] = (p + rho)*W*W - p
    def con2prim(self, cons, prim, argv=None):
        D, S, E = cons
        rho0, ux, eps = prim
        
        con2prim_kernel(self.eos, D, S, E, rho0, ux, eps)
    
    def speeds(self, prim, cons, char, argv=None):
        rho0, ux, eps = prim
        
        W, v = W_v_from_ux(ux)
        cs = self.eos.csound(rho0, eps)
        
        char[0,:] = (v - cs)/(1 - v*cs)
        char[1,:] = v
        char[2,:] = (v + cs)/(1 + v*cs)
    def fluxes(self, prim, cons, flux, argv=None):
        rho0, ux, eps = prim
        D, S, E = cons
        
        W, v = W_v_from_ux(ux)
        p = self.eos.press(rho0, eps)
        
        flux[0,:] = D*v
        flux[1,:] = S*v + p
        flux[2,:] = (E + p)*v
    def sources(self, prim, cons, source, argv=None):
        source[:] = 0.

### Relativistic Sod test

Here we implement and run the relativistic version of the Sod test. We also implement a new class to store the results of the numerical calculation.

In [ ]:
class RelSodInitialData(hrsc.InitDataGeneric):
    def apply(self, grid, prim, argv=None):
        """
        Set up the value of the primitive variables.
        As optional argument we take the grid point location.
        """
        rho, ux, eps = prim
        rho[:] = np.where(grid.xc < 0, 1.0, 0.125)
        ux[:] = 0.
        eps[:] = np.where(grid.xc < 0, 2.5, 2.0)

Here we solve the equations. This code is very similar to the code above for the Newtonian case, with minor modifications.

In [ ]:
grid_rel_sod = hrsc.make_uniform_grid(-0.5, 0.5, 100)
eos_rel_sod = RelIdealGas(gamma=1.4)
claw_rel_sod = RelEuler(eos_rel_sod)
ic_rel_sod = RelSodInitialData()
bc_rel_sod = hrsc.BoundaryStatic()
solver_rel_sod = hrsc.KurganovTadmorSolver(grid_rel_sod, claw_rel_sod, ic_rel_sod, bc_rel_sod)

In [ ]:
sol_rel_sod = run_solver(solver_rel_sod, CFL=CFL, t_start=0, t_end=0.4, nframes=10)
sol_rel_sod.times = np.array(sol_rel_sod.times)
for frame in sol_rel_sod.frames:
    frame["p"] = eos_rel_sod.press(frame["rho0"], frame["eps"])
    frame["W"], frame["v"] = W_v_from_ux(frame["ux"])

In [ ]:
@interact(time=FloatSlider(value=0, min=sol_rel_sod.times[0], max=sol_rel_sod.times[-1],
                           step=sol_rel_sod.times[1] - sol_rel_sod.times[0],
                           continuous_update=True))
def plot_sod(time):
    idx = np.argmin(np.abs(sol_rel_sod.times - time))
    plt.figure(figsize=[10,7])
    plt.title("Time: {:.3f}".format(sol_rel_sod.times[idx]))
    plt.plot(grid_rel_sod.xc, sol_rel_sod.frames[idx]["rho0"], ".", label=r"$\rho_0$", color="blue")
    plt.plot(grid_rel_sod.xc, sol_rel_sod.frames[idx]["v"], ".", label=r"$v$", color="red")
    plt.plot(grid_rel_sod.xc, sol_rel_sod.frames[idx]["p"], ".", label=r"$p$", color="green")
    plt.xlim(-0.5, 0.5)
    plt.ylim(-0.1, 1.1)
    plt.legend()
    plt.show()

As for the Newtonian case, it is possible to compute the exact solution for this problem also for a relativistic fluid.

In [ ]:
exact = np.load("./data/rel_sod_t0p4.npz")
plt.figure(figsize=[10,7])
plt.title("Time: 0.4")
plt.plot(exact["x"], exact["rho0"], "-", label=r"$\rho_0$ exact", color="blue")
plt.plot(exact["x"], exact["v"], "-", label=r"$v$ exact", color="red")
plt.plot(exact["x"], exact["p"], "-", label=r"$p$ exact", color="green")
plt.plot(grid_rel_sod.xc, sol_rel_sod.frames[-1]["rho0"], ".", label=r"$\rho_0$ numerical", color="blue")
plt.plot(grid_rel_sod.xc, sol_rel_sod.frames[-1]["v"], ".", label=r"$v$ numerical", color="red")
plt.plot(grid_rel_sod.xc, sol_rel_sod.frames[-1]["p"], ".", label=r"$p$ numerical", color="green")
plt.legend()

### Relativistic blast wave

This is an example of a strong relativistic blastwave. The adiabatic index is set to $5/3$ and the initial data is

$$
    (n_L, v_L, \epsilon_L) = (0.001, 0, 1500), \quad
    (n_R, v_R, \epsilon_R) = (0.001, 0, 0.015)
$$

Note that $\epsilon_L \gg 1$, meaning that the fluid on the left side is relativistically hot. The exact solution consist of a transonic rarefaction wave, a contact discontinuity and a shock. The region between the contact wave and the shock, the blast wave, is relativistically contracted into a narrow shell that is extremelly challenging to resolve numerically.

In [ ]:
class RelBlastInitialData(hrsc.InitDataGeneric):
    def apply(self, grid, prim, argv=None):
        """
        Set up the value of the primitive variables.
        As optional argument we take the grid point location.
        """
        rho, ux, eps = prim
        rho[:] = np.where(grid.xc < 0, 0.001, 0.001)
        ux[:] = 0.
        eps[:] = np.where(grid.xc < 0, 1500, 0.015)

For this test problem we use a high-resolution grid with 1000 grid points. We also use a more dissipative flux formula: Rusanov instead of HLLE.

In [ ]:
grid_rel_blast = hrsc.make_uniform_grid(-0.5, 0.5, 1000)
eos_rel_blast = RelIdealGas(gamma=5./3.)
claw_rel_blast = RelEuler(eos_rel_blast)
ic_rel_blast = RelBlastInitialData()
bc_rel_blast = hrsc.BoundaryStatic()
solver_rel_blast = hrsc.KurganovTadmorSolver(grid_rel_blast, claw_rel_blast, ic_rel_blast, bc_rel_blast,
                                            flux_formula="Rusanov")

In [ ]:
sol_rel_blast = run_solver(solver_rel_blast, CFL=CFL, t_start=0, t_end=0.4, nframes=10)
sol_rel_blast.times = np.array(sol_rel_blast.times)
for frame in sol_rel_blast.frames:
    frame["p"] = eos_rel_blast.press(frame["rho0"], frame["eps"])
    frame["W"], frame["v"] = W_v_from_ux(frame["ux"])

In [ ]:
@interact(time=FloatSlider(value=0, min=sol_rel_blast.times[0], max=sol_rel_blast.times[-1],
                           step=sol_rel_blast.times[1] - sol_rel_blast.times[0],
                           continuous_update=True))
def plot_blast(time):
    idx = np.argmin(np.abs(sol_rel_blast.times - time))
    plt.figure(figsize=[10,7])
    plt.title("Time: {:.3f}".format(sol_rel_blast.times[idx]))
    plt.plot(grid_rel_blast.xc, 100*sol_rel_blast.frames[idx]["rho0"], ".",
             label=r"$100\times\rho_0$", color="blue")
    plt.plot(grid_rel_blast.xc, sol_rel_blast.frames[idx]["v"], ".",
             label=r"$v$", color="red")
    plt.plot(grid_rel_blast.xc, sol_rel_blast.frames[idx]["p"], ".",
             label=r"$p$", color="green")
    plt.xlim(-0.5, 0.5)
    plt.ylim(-0.1, 1.1)
    plt.legend()
    plt.show()

In [ ]:
exact = np.load("./data/rel_blast_t0p4.npz")
plt.figure(figsize=[10,7])
plt.title("Time: 0.4")
plt.plot(exact["x"], 1e2*exact["rho0"], "-", label=r"$100 \times \rho_0$ exact", color="blue")
plt.plot(exact["x"], exact["v"], "-", label=r"$v$ exact", color="red")
plt.plot(exact["x"], exact["p"], "-", label=r"$p$ exact", color="green")
plt.plot(grid_rel_blast.xc, 1e2*sol_rel_blast.frames[-1]["rho0"], ".",
         label=r"$\rho_0$ numerical", color="blue")
plt.plot(grid_rel_blast.xc, sol_rel_blast.frames[-1]["v"], ".",
         label=r"$v$ numerical", color="red")
plt.plot(grid_rel_blast.xc, sol_rel_blast.frames[-1]["p"], ".",
         label=r"$p$ numerical", color="green")
plt.ylim(-0.05, 1.05)
plt.legend()